# Simulation post-processing

<!-- SUMMARY: The Simulation post-processing is illustrated through an Upscaling capability -->

<!-- CATEGORY: Data_Base_Management -->

This test is meant to demonstrate the tool for post-processing simulation results.

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import numpy as np
import matplotlib.pyplot as plt
import gstlearn.plot as gp
import scipy.sparse as sc

gdoc.setNoScroll()

We construct several simulations independently. Here, the manner these simulations are constrcuted has no interest and we simply sample distributions (Uniform or Gaussian) with various parameters. We construct three simulations (called *SimuA*, *SimuB* and *SimuC*) with different occurence numbers (3, 2 and 2).

Then, for each cell of the output grid:
- a search for the samples belonging to the cell is performed
- a particular transformation (calculating their *sum* and their *standard deviation*) is applied to the outcomes of these samples
- the transformed values of these samples are upscaled (using the *mean*) to one single multivariate per cell
- Some statistics (*mean* and *variance*) are calculated on the results of the transformation per cell.

Creating a point Data Base

In [ ]:
nech=10
db = gl.Db.createFillRandom(ndat=nech, ndim=2, nvar=0, seed = 3143233)
err = db.addColumns(np.arange(0,nech),"rank")

Creating a Grid Data Base covering the same area

In [ ]:
nx = 5
dx = 1 / nx
x0 = dx / 2
dbgrid = gl.DbGrid.create(nx=[nx,nx], dx=[dx,dx], x0=[x0,x0])

Adding a first set of 3 simulation outcomes (sampled from Normal distribution)

In [ ]:
nsimuA = 3
uid = db.addColumnsByConstant(nsimuA, 0., "SimuA")
for i in range(nsimuA):
    vec = gl.VectorHelper.simulateGaussian(nech)
    db.setColumnByUID(vec, uid + i)

Adding a second set of 2 simulations (sampled from Uniform distribution between 100 and 200)

In [ ]:
nsimuB = 2
uid = db.addColumnsByConstant(nsimuB, 0., "SimuB")
for i in range(nsimuB):
    vec = gl.VectorHelper.simulateUniform(nech, 100., 200.)
    db.setColumnByUID(vec, uid + i)

Adding a third set of 2 simulations (sampled from Uniform distribution between -4 and -2)

In [ ]:
nsimuC = 2
uid = db.addColumnsByConstant(nsimuC, 0., "SimuC")
for i in range(nsimuC):
    vec = gl.VectorHelper.simulateUniform(nech, -4., -2.)
    db.setColumnByUID(vec, uid + i)

In the following paragraph, we provide the complete dump of the contents of the Data Base to better understand the next steps.

In [ ]:
dbfmt = gl.DbStringFormat.createFromFlags(flag_resume = False, flag_vars=False, flag_array=True)
gl.OptCst.defineByKey("NTROW",-1)
gl.OptCst.defineByKey("NTCOL",-1)
db.display(dbfmt)

Display of the samples and overlay the grid of cells

In [ ]:
gp.cell(dbgrid)
gp.literal(db, name="rank", s=100)
plt.show()

We perform the simulations post-processing as described in the introduction.

In this first application, we bypass any transformation function.

A particular attention is brought to the cell (ranked 4 [1-based]) in order to check the work flow applied to the information.

In [ ]:
err = gl.simuPost(db, dbgrid, ["SimuA*", "SimuB*", "SimuC*"], flag_match=False, 
                  upscale = gl.EPostUpscale.MEAN,
                  stats = [gl.EPostStat.MEAN, gl.EPostStat.VAR], 
                  check_targets=[4], verbose=True, namconv=gl.NamingConvention("Post1"))

In [ ]:
dbgrid

In [ ]:
gp.literal(dbgrid, name="Post1.Var1.Variance", marker="")
gp.cell(dbgrid)
gp.literal(db, name="rank", s=100)
plt.show()

In this second trial, we use *simuPostDemo* where a stransformation has been embedded in the procedure: this transformation calculates the *sum* and the *standard deviation* of the simulation outcomes for eachsample of the target cell.
For comparison sake, we still keep the focus on the same cell.

In [ ]:
err = gl.simuPostDemo(db, dbgrid, ["SimuA*", "SimuB*", "SimuC*"], flag_match=False, 
                      upscale = gl.EPostUpscale.MEAN,
                      stats = [gl.EPostStat.MEAN, gl.EPostStat.VAR], 
                      check_targets=[4], verbose=True, namconv=gl.NamingConvention("Post2"))

In [ ]:
gp.literal(dbgrid, name="Post2.Var1.Variance", marker="")
gp.cell(dbgrid)
gp.literal(db, name="rank", s=100)
plt.show()

In the next paragraph, we perform the simulation post-processing stage in place. This means that we do not provide any output Db and the Upscaling phase does not take place (however note that the upscaling rule *must* not be left empty). The results are stored in the Input Db.

In [ ]:
err = gl.simuPost(db, None, ["SimuA*", "SimuB*", "SimuC*"], flag_match=False, 
                  upscale = gl.EPostUpscale.UNKNOWN,
                  stats = [gl.EPostStat.MEAN, gl.EPostStat.VAR], 
                  check_targets=[4], verbose=True, namconv=gl.NamingConvention("Post1"))

In [ ]:
db.display(dbfmt)

Same operation with a (built-in) transformation.

In [ ]:
err = gl.simuPostDemo(db, None, ["SimuA*", "SimuB*", "SimuC*"], flag_match=False, 
                      upscale = gl.EPostUpscale.UNKNOWN,
                      stats = [gl.EPostStat.MEAN, gl.EPostStat.VAR], 
                      check_targets=[4], verbose=True, namconv=gl.NamingConvention("Post2"))

In [ ]:
db.display(dbfmt)